In [ ]:
from langgraph.graph import StateGraph , START , END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing_extensions import TypedDict
from dotenv import load_dotenv
import os
gemini_api_key = os.getenv("GEMINI")
class Blogstate(TypedDict):
    Title : str
    Outline : str
    Content : str
    
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash" , api_key=gemini_api_key)    

In [7]:
def outline(state:Blogstate)->Blogstate:
    title = state['Title']
    prompt = f"generate the outline of the project name {title}"
    state['Outline'] = llm.invoke(prompt)
    return state

def content(state:Blogstate)->Blogstate:
    title = state['Title']
    outline = state['Outline']
    prompt = f"generate the content of the project name {title} and outline is {outline}"
    state['Outline'] = llm.invoke(prompt)
    return state
    

In [8]:
graph = StateGraph(Blogstate)

graph.add_node("outline" , outline)
graph.add_node("content",content)

graph.add_edge(START , "outline")
graph.add_edge("outline" , "content")
graph.add_edge("content","END")
workflow = graph.compile()
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png()) 
initial_state = {
      "Title" : "ai agent for booking",
    "Outline" : "",
    "Content" : ""
}
final_state = workflow.invoke(initial_state)
print(final_state)

ValueError: Found edge ending at unknown node `END`